In [1]:
%pip install -qU openai marvin
%pip install -qU "psycopg[binary]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Getting ready

Begin by:
1. creating a new directory `data/<jurisdiction>` and populate with one or more
docx files containing the jurisdiction's municipal code
2. run `scripts/convert_docx.sh` to convert those files into a single text file
3. make a copy of `notebooks/template-workflow.ipynb` to `notebooks/<jurisdiction>.ipynb`
and continue processing in that notebook

In [2]:
import sys
sys.path.insert(0, '..')

## set up auto-reloading for development
%reload_ext autoreload
%autoreload 2

## Specify heading patterns

Replace the `jurisdiction_headings` dict with examples from your jurisdiction

In [3]:
from muni.code import Jurisdiction

In [4]:
from openai import OpenAI
client = OpenAI()

response = client.embeddings.create(
  input="It was the best of times, it was the worst of times.",
  model="text-embedding-3-small"
)

print(response.data[0].embedding)

[-0.037202694, 0.003527262, 0.034943096, 0.014647048, 0.044680893, -0.011970499, 0.023793047, 0.011735124, -0.05073339, 0.028971296, 0.029132696, -0.035212096, -0.014055248, -0.0095427735, 0.040080994, 0.012690073, -0.0033423244, 0.027975995, -0.004572999, 0.041963995, -0.004418324, 0.027733896, 0.019421797, 0.014351148, -0.0064324616, 0.019435247, 0.012737148, -0.0016610748, 0.09086819, 0.039381593, 0.008446598, -0.031257797, 0.007895149, 0.014351148, -0.036449496, 0.033598095, -0.0052286866, -0.042636495, 0.026281295, -0.036234293, 0.008090174, 0.023685446, 0.015252298, 0.04796269, -0.017323598, -0.06181619, -0.06681959, -0.035750095, 0.040080994, 0.032118596, -0.031957194, 0.030047296, -0.015723048, 0.0028732559, 0.0111029735, -0.023389546, -0.010161473, 0.009314124, 0.026227497, -0.06509799, 0.044169795, 0.000999503, 0.022851547, 0.014539448, 0.014149398, -0.05638239, 0.0006897327, 0.0111029735, -0.004623437, -0.0019098998, 0.08575719, 0.014647048, -0.042770993, -0.027074846, 0.033

In [5]:
heading_examples = {
    1: ["TITLE 1\nGENERAL PROVISION\n",
        "TITLE 2\nCITY GOVERNMENT AND ADMINISTRATION\n",
        "TITLE 3\nREVENUE AND FINANCE\n",
        ],
    2: ["CHAPTER 1-4\nCODE ADOPTION - ORGANIZATION\n",
        "CHAPTER 1-8\nCITY SEAL AND FLAG\n",
        "CHAPTER 1-12\nCITY EMBLEMS\n",
        ],
    3: ["1-4-010 Municipal Code of Chicago adopted.\n",
        "2-1-020 Code to be kept up-to-date.\n",
        "3-4-030 Official copy on file.\n",
        ],
}

In [6]:
from muni.code import infer_heading_patterns, infer_level_names

In [7]:
## Verify that the regular expressions matching outline levels look okay
heading_patterns = infer_heading_patterns(heading_examples)
for level, pattern in heading_patterns.items():
    print(f"{level}: r'{pattern.regex}'")

print()

## Verify that the names of the sections look okay
level_names = infer_level_names(heading_patterns)
for level, name in level_names.items():
    print(f"{level}: {name}")

1: r'^TITLE (\d+)$'
2: r'^CHAPTER (\d+-\d+)$'
3: r'^(\d+-\d+-\d+) (.+)\.$'

1: Title
2: Chapter
3: Section


## Specify the parameters of the jurisdiction and parse the code

In [8]:
place = Jurisdiction(
    name="Chicago Mini",
    title="Municipal Code of Chicago",
    patterns=heading_patterns,
    level_names=level_names,
    source_local="../data/chicago-mini/code.txt",
    source_url="https://www.chicago.gov/city/en/depts/doit/supp_info/municipal_code.html",
)

place.parse()
place.chunkify(1000)

In [9]:
## Verify that the distribution of paragraphs and chunks looks okay
place.summarize()

    H1 Title 1 (GENERAL PROVISIONS): 1 chunks, 12 paragraphs, 503 characters
        H2 Chapter 1-4 (CODE ADOPTION - ORGANIZATION*): 1 chunks, 1 paragraphs, 178 characters
            H3 Section 1-4-010 (Municipal Code of Chicago adopted): 1 chunks, 2 paragraphs, 478 characters
            H3 Section 1-4-020 (Adoption of chapter and section numbers): 1 chunks, 2 paragraphs, 463 characters
            H3 Section 1-4-030 (Effective date): 1 chunks, 2 paragraphs, 129 characters
            H3 Section 1-4-040 (Administrative copies - City clerk powers and duties): 1 chunks, 3 paragraphs, 514 characters
            H3 Section 1-4-050 (Code revisions - Publication): 1 chunks, 2 paragraphs, 195 characters
            H3 Section 1-4-060 (Publication of Code - Conditions): 1 chunks, 2 paragraphs, 322 characters
            H3 Section 1-4-070 (Distribution of Code): 1 chunks, 3 paragraphs, 615 characters
            H3 Section 1-4-080 (Numbering of Code sections - References to former Code provi

## Upload data to the database

In [10]:
from muni.code import upload

db = {'dbname': 'muni',
      'user': 'muni',
      'password': '',
      'host': 'localhost',
      'port': 5432}

upload(db, place)

In [11]:
from muni.code import upload_embeddings, refresh_views

upload_embeddings(db, place)
refresh_views(db)

## Find associations among sections

In [12]:
from muni.code import find_associations

find_associations(db, place)
# TODO: changing DB schema

## Basic queries

In [13]:
from muni.code import simple_semantic_query

queries = ['Does the municipal code contain provisions restricting the use of drug paraphernalia?']

for query in queries:
    print(f"Query: {query}")
    results = simple_semantic_query(db, place, query, limit=20)
    for result in results:
        print(result)

Query: Does the municipal code contain provisions restricting the use of drug paraphernalia?
(1552, 'Title 7 HEALTH AND SAFETY\nChapter 7-24 DRUGS AND NARCOTICS\nSection 7-24-090   Sale of certain substances\n...\nArticle II. Drug Paraphernalia\n...\n')
(1571, 'Title 7 HEALTH AND SAFETY\nChapter 7-24 DRUGS AND NARCOTICS\nSection 7-24-091 Possession or delivery\n...\n   1.   Except as authorized by law, any\nperson who delivers, furnishes, transfers, or possesses with intent to\ndeliver, furnish or transfer, drug paraphernalia, as defined in Section\n720 ILCS 600/2, subparagraph (d)(5) of the Drug Paraphernalia Control\nAct, and including glass tubing designed and utilized for the ingestion\nof crack or cocaine, knowing, or under circumstances where one\nreasonably should know, that such drug paraphernalia or glass tubing\nwill be used to plant, propagate, cultivate, sow, harvest, compound,\nconvert, produce, process, prepare, test, analyze, pack, repack, store,\ncontain, conceal, injec

In [14]:
from muni.code import extract_keywords, simple_full_text_query

## FIXME: this doesn't work well because extract_keywords() returns too many keywords
# queries = [' '.join(extract_keywords(query)) for query in queries]

queries = ['drug paraphernalia']

for query in queries:
    print(f"Query: {query}")
    results = simple_full_text_query(db, place, query, limit=20)
    for result in results:
        print(result)


Query: drug paraphernalia
(1572, 'Title 7 HEALTH AND SAFETY\nChapter 7-24 DRUGS AND NARCOTICS\nSection 7-24-092 Manufacture\n...\n   Except as authorized by law, any person who\nmanufactures, with intent to deliver, furnish, or transfer drug\nparaphernalia knowing, or under circumstances where one reasonably\nshould know, that it will be used to plant, propagate, cultivate, grow,\nharvest, manufacture, compound, convert, produce, process, prepare,\ntest, analyze, pack, repack, store, contain, conceal, ingest, inhale or\notherwise introduce into the human body cocaine, cocaine base, heroin,\nphencyclidine, or methamphetamine in violation of the Illinois\nControlled Substances Act shall be fined $1,000.00, or punished by\nimprisonment for a period of six months, or by both such fine and\nimprisonment.\n(Added Coun. J. 5-12-99, p. 3327)\n...\n')
(1552, 'Title 7 HEALTH AND SAFETY\nChapter 7-24 DRUGS AND NARCOTICS\nSection 7-24-090   Sale of certain substances\n...\nArticle II. Drug Paraphe

In [15]:
from muni.code import hybrid_query

queries = ['Does the municipal code contain provisions restricting the use of drug paraphernalia?']

for query in queries:
    print(f"Query: {query}")
    results = hybrid_query(db, place, query, limit=20)
    for result in results:
        print(result)

Query: Does the municipal code contain provisions restricting the use of drug paraphernalia?
(1552, Decimal('0.01639344262295081967'), 'Title 7 HEALTH AND SAFETY\nChapter 7-24 DRUGS AND NARCOTICS\nSection 7-24-090   Sale of certain substances\n...\nArticle II. Drug Paraphernalia\n...\n')
(1571, Decimal('0.01612903225806451613'), 'Title 7 HEALTH AND SAFETY\nChapter 7-24 DRUGS AND NARCOTICS\nSection 7-24-091 Possession or delivery\n...\n   1.   Except as authorized by law, any\nperson who delivers, furnishes, transfers, or possesses with intent to\ndeliver, furnish or transfer, drug paraphernalia, as defined in Section\n720 ILCS 600/2, subparagraph (d)(5) of the Drug Paraphernalia Control\nAct, and including glass tubing designed and utilized for the ingestion\nof crack or cocaine, knowing, or under circumstances where one\nreasonably should know, that such drug paraphernalia or glass tubing\nwill be used to plant, propagate, cultivate, sow, harvest, compound,\nconvert, produce, process,

## Report generation

In [16]:
from muni.code import Report
from IPython.display import display, Markdown

query = 'Does the municipal code contain provisions restricting the use of drug paraphernalia?'

report = Report(db, place, query)

display(Markdown(str(report)))

Found 7 relevant context blocks for query 'Does the municipal code contain provisions restricting the use of drug paraphernalia?' in Chicago Mini


### Query: Does the municipal code contain provisions restricting the use of drug paraphernalia?

### Short answer:

yes

### Full Answer:

Yes, the municipal code of Chicago contains provisions restricting the use of drug paraphernalia. According to Section 7-24-091, it is illegal to deliver, furnish, transfer, or possess with intent to deliver drug paraphernalia, such as glass tubing used for ingesting crack or cocaine, knowing that it will be used with a controlled substance in violation of the Illinois Controlled Substances Act. Violations can result in a fine of $2,000, imprisonment for six months, or both (Title 7, Chapter 7-24, Section 7-24-091). Additionally, all such drug paraphernalia is subject to seizure and forfeiture as per Section 7-24-096."}

### Supporting context:

#### Excerpt:

Title 7 HEALTH AND SAFETY
Chapter 7-24 DRUGS AND NARCOTICS
Section 7-24-091 Possession or delivery
...
   1.   Except as authorized by law, any
person who delivers, furnishes, transfers, or possesses with intent to
deliver, furnish or transfer, drug paraphernalia, as defined in Section
720 ILCS 600/2, subparagraph (d)(5) of the Drug Paraphernalia Control
Act, and including glass tubing designed and utilized for the ingestion
of crack or cocaine, knowing, or under circumstances where one
reasonably should know, that such drug paraphernalia or glass tubing
will be used to plant, propagate, cultivate, sow, harvest, compound,
convert, produce, process, prepare, test, analyze, pack, repack, store,
contain, conceal, inject, ingest, inhale, or otherwise introduce into
the human body a controlled substance in violation of the Illinois
Controlled Substances Act, codified at 720 ILCS 570/100, et seq., shall
be fined $2,000.00, or punished by imprisonment for a period of six
months, or punished by both such fine and imprisonment.
   2.   Except as authorized by law, any
person who possesses self-sealing plastic bags under two inches in
either height or width, knowing that such items will be or are being
used to package for transfer, delivery and/or storage of a controlled
substance in violation of the Illinois Controlled Substances Act,
codified at 720 ILCS 570/100, et seq., shall be fined up to $1,500
depending on the quantity of such items in their possession.
(Added Coun. J. 5-12-99, p. 3327; Amend Coun. J. 11-5-08, p. 43597,
§ 1; Amend Coun. J. 3-18-09, p. 56051; Amend Coun. J. 5-9-12, p.
27485, § 155; Amend Coun. J. 11-8-12, p. 38872, § 141)
...


#### Excerpt:

Title 7 HEALTH AND SAFETY
Chapter 7-24 DRUGS AND NARCOTICS
Section 7-24-096 Seizure and forfeiture
...
   All drug paraphernalia defined in Section 720 ILCS
600/2, subparagraph (5), and including glass tubing utilized for the
ingestion of cocaine or crack cocaine, is subject to forfeiture and may
be seized by any peace officer. The seizure and forfeiture shall be made
in accordance with rules issued by the superintendent of police or his
designee.
(Added Coun. J. 5-12-99, p. 3327)
...


### Other context:

#### Excerpt:

Title 7 HEALTH AND SAFETY
Chapter 7-24 DRUGS AND NARCOTICS
Section 7-24-090   Sale of certain substances
...
Article II. Drug Paraphernalia
...


#### Excerpt:

Title 7 HEALTH AND SAFETY
Chapter 7-24 DRUGS AND NARCOTICS
Section 7-24-090 Sale of certain substances
...
   No person shall knowingly sell or offer for sale,
deliver or give away to any person under 17 years of age, unless upon
the written order of parent or guardian, any substances containing any
of the following volatile solvents, where the seller, offerer or
deliverer knows or has reason to believe that the substance will be used
for the purpose of inducing symptoms of intoxication, elation,
dizziness, paralysis, irrational behavior, or in any manner change,
distort or disturb the audio, visual or mental processes:
   Toluol, hexane, trichloroethylene, acetone, toluene,
ethyl acetate, methyl ethyl ketone, trichloroethane, isopropanol, methyl
isobutyl ketone, methyl cellosolve acetate, cyclohexanone, or any other
substance which will induce symptoms of intoxication, elation,
dizziness, paralysis, irrational behavior, or in any manner change,
distort or disturb the audio, visual or mental processes.
   Any person violating any of the provisions of this
section shall be fined not less than $25.00 nor more than $200.00 for
each offense.
(Prior code § 97-7.2)
ARTICLE II. DRUG PARAPHERNALIA* (7-24-091 et seq.)
* Editor's note - Coun. J. 5-12-99, p. 3327, renumbered Articles
II - VI as Articles III - VIII to accommodate inclusion of
§§ 7-24-091 - 7-24-097, as Art. II.
...


#### Excerpt:

Title 7 HEALTH AND SAFETY
Chapter 7-24 DRUGS AND NARCOTICS
Section 7-24-092 Manufacture
...
   Except as authorized by law, any person who
manufactures, with intent to deliver, furnish, or transfer drug
paraphernalia knowing, or under circumstances where one reasonably
should know, that it will be used to plant, propagate, cultivate, grow,
harvest, manufacture, compound, convert, produce, process, prepare,
test, analyze, pack, repack, store, contain, conceal, ingest, inhale or
otherwise introduce into the human body cocaine, cocaine base, heroin,
phencyclidine, or methamphetamine in violation of the Illinois
Controlled Substances Act shall be fined $1,000.00, or punished by
imprisonment for a period of six months, or by both such fine and
imprisonment.
(Added Coun. J. 5-12-99, p. 3327)
...


#### Excerpt:

Title 7 HEALTH AND SAFETY
Chapter 7-24 DRUGS AND NARCOTICS
Section 7-24-040 Reserved
...
Editor's note - Coun. J. 10-3-01, p. 68139, § 3, repealed
§ 7-24-040, which pertained to habit-forming drugs.
...


#### Excerpt:

Title 7 HEALTH AND SAFETY
Chapter 7-24 DRUGS AND NARCOTICS
Section 7-24-060 Reserved
...
Editor's note - Coun. J. 10-3-01, p. 68139, § 3, repealed
§ 7-24-060, which pertained to opium smoking rooms.
...




## Upload results to database

In [17]:
from muni.code import upload_report

upload_report(db, report)